# 8760 Notebook

In this notebook, we’ll generate an 8760 timeseries and insert 1-in-X events into it. We'll demonstrate how to insert either the minimum, maximum, or both types of 1-in-X events. You can choose specific times to insert the events, or let the backend automatically determine the median timing based on when those events occurred in the original timeseries.

### Imports

In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from phase3 import *
from climakitae.util.warming_levels import get_gwl_at_year, get_year_at_gwl
random.seed(35)

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

### Global Variables

In [2]:
# Putting locations into a DataFrame
locations = {
    "Goleta": (34.486652, -119.894310),
    "Isla Vista": (34.427741, -119.855950),
    "Santa Barbara Mountains": (34.518453, -119.799645),
    "Valencia": (34.432468, -118.571098),
    "Castaic": (34.492141, -118.605053),
    "Lebec": (34.680102, -118.789246),
}

df = pd.DataFrame.from_dict(locations, orient='index', columns=['lat', 'lon']).reset_index()
df.rename(columns={'index': 'location'}, inplace=True)

# Santa Barbara area
sb = df.iloc[:3][['lat', 'lon']]
sb_locs = ['Goleta', 'Isla Vista', 'Santa Barbara Mountains']

# Santa Clarita area
sc = df.iloc[3:6][['lat', 'lon']]
sc_locs = ['Valencia', 'Castaic', 'Lebec']

In [3]:
# Set params for 1-in-X calculations
one_in_x = np.array([10, 100])
event_duration = (1, 'day')
distr = 'gev'

## Introduction of Custom WLs

In this version, we will introduce custom warming levels, and a tool to help you find a custom WL appropriate for your planning processes.

Below, you'll see how to use two functions that translate between years and GWLs.

**An important note:**
In order to use the SSP parameter for the following functions, the SSP values need to be one (or more) of the following:

```
"Historical", "SSP 1-1.9", "SSP 1-2.6", "SSP 2-4.5", "SSP 3-7.0", "SSP 5-8.5", or "all"
```

The bounds of the potential warming levels are from 0 to 5.702 for WRF, and the bounds of the valid years are from 1950-2100.

#### Examples

In this first example, we will use `get_gwl_at_year` to find the WL associated with a given year for SSP 3-7.0.

In [ ]:
get_gwl_at_year(2020, ssp="SSP 3-7.0")

You can additionally pass in a year and get back GWLs across all SSPs.

In [ ]:
get_gwl_at_year(2050, ssp="all")

If you're interested in knowing what year is associated with a given warming level, you can do the reverse, with `get_year_at_gwl`.

In [ ]:
get_year_at_gwl(2.1, ssp="SSP 3-7.0")

And the same `all` SSP argument if you wanted to see all SSPs.

In [ ]:
get_year_at_gwl(2.1, ssp="all")

You'll notice that the above DataFrame returns NaNs for certain SSPs, this is because those SSPs don't have any GCMs that reach that given level of warming.

In [ ]:
# Example with a really high WL
get_year_at_gwl(6.7, ssp="all")

From here, you can decide how you'd like to aggregate the WL values across the SSPs from the returned table from `get_gwl_at_year`. Then, you can pass in that WL value to the following `retrieve_data` function to get custom WL data!

## Reading/Loading WL Data

In [4]:
### Retrieving saved files for SB/SC locations, or generating the data from scratch if not available locally
sb_files = [
#     'custom-wls/one_in_10-100_temperature_raw_data_34495N_-119882W.nc',
#     'custom-wls/one_in_10-100_temperature_raw_data_34434N_-11987W.nc',
#     'custom-wls/one_in_10-100_temperature_raw_data_3452N_-119782W.nc'
]
sc_files = [
#     'one_in_10-100_temperature_raw_data_34425N_-118559W.nc',
#     'one_in_10-100_temperature_raw_data_34492N_-118616W.nc',
#     'one_in_10-100_temperature_raw_data_34695N_-118788W.nc',
]

# Pass in custom parameters
clean_raw_data_sb = retrieve_data(
    sb_locs, 
    sb, 
    sb_files, 
    'SB', 
    hour_offset=7, # Offset hours from UTC to a different timezone (-7 to PST)
    warming_level=3.14159 # Pass in a custom WL here!
)
clean_raw_data_sc = retrieve_data(
    sc_locs, 
    sc, 
    sc_files, 
    'SC', 
    hour_offset=7, 
    warming_level=2.71818
)

Attempting to load saved SB files...
Falling back to cava_data for SB due to error: no files to open
Using `cava_data` to fetch SB data instead...
Calculating 1-in-10 year, 1-day Max Air Temperature at 2m for all seasons for Warming Level 3.142°C and 1-in-100 year, 1-day Max Air Temperature at 2m for all seasons for Warming Level 3.142°C
--- Selecting Data Points --- 

Selecting data for (34.486652, -119.89431)
Selecting data for (34.427741, -119.85595)
Selecting data for (34.518453, -119.799645)

--- Loading Data into Memory ---

Point (34.49457931518555, -119.88214111328125)
Processing data to read 8.02 MB of data into memory... 
[########################################] | 100% Completed | 72.08 s
Complete!
Point (34.433998107910156, -119.87033081054688)
Processing data to read 8.02 MB of data into memory... 
[########################################] | 100% Completed | 75.64 s
Complete!
Point (34.5202751159668, -119.78153991699219)
Processing data to read 8.02 MB of data into memor

### Calculating 1-in-X events

In [5]:
%%time
# Calculating the 1-in-X return values, and lower + upper CIs, for SB locations
all_one_in_x_sb = get_one_in_x(clean_raw_data_sb, one_in_x, event_duration, distr, 'max')
calc_data_sb = all_one_in_x_sb['return_value']

/home/jovyan/src/climakitae/climakitae/explore/threshold_tools.py:231: UserWarning: 

No NaN values will be dropped from the block maxima DataArray.
Please inspect the data and handle NaN values appropriately before proceeding.
  warnings.warn(


ValueError: all input arrays must have the same shape

In [6]:
%%time
# Calculating the 1-in-X return values, and lower + upper CIs, for SC locations
all_one_in_x_sc = get_one_in_x(clean_raw_data_sc, one_in_x, event_duration, distr, 'max')
calc_data_sc = all_one_in_x_sc['return_value']

CPU times: user 18 s, sys: 0 ns, total: 18 s
Wall time: 17.9 s


In [ ]:
# Plotting the 1-in-X values by simulation for Santa Barbara locations
plot_retvals(calc_data_sb)

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Clarita locations
plot_retvals(calc_data_sc)

In [ ]:
# Finding the median of the above plots along simulation dimension
plot_med_val_by_locs(calc_data_sb)

In [ ]:
plot_med_val_by_locs(calc_data_sc)

# Inserting `time` amount of 1-in-X timesteps into 8760s

### Setting a number of +/- days for `ramp up` and `ramp down`, creating modified 8760s, and plotting them 

In [ ]:
# Setting param for number of +/- number of days to 'ramp up' and 'ramp down' by
t = 14

#### Here, we will look into how you can create a `modified8760` using the calculated median times to insert these 1-in-X events, and using the times that a user passes in.

#### 1. Using calculated median times to insert 1-in-X events into

In [ ]:
# Create a modified 8760 that injects 1-in-X events into the median times these events occur by location
modified8760_sb, insert_times = create_modified_8760(clean_raw_data_sb, all_one_in_x_sb, t)
plot_modified8760s(modified8760_sb, insert_times, t)

#### 2. Insert 1-in-X events at times that users pre-define

In [ ]:
# This creates a dummy DataArray that follows the data shape needed to specify custom times
dummy_times = create_empty_da(all_one_in_x_sb['return_value'], keep_dims=['location', 'one_in_x'])

In [ ]:
# Here, you can manually change the values from `custom_times` to set specific times for these `one_in_x` events to occur.
# `updates` can either be a list of dictionaries (if you're looking to set these different events at different times, or you can
# set `updates` to a singular number to have all the events be inserted at the same time

updates = [
    {'location': 'Goleta', 'one_in_x': 10, 'value': 1000},
    {'location': 'Isla Vista', 'one_in_x': 100, 'value': 1000}
]

# OR YOU CAN DO
updates = 3015

custom_times = set_custom_times(dummy_times, updates)

In [ ]:
# Calculates the modified 8760 with 1-in-X events inserted at custom times
modified8760_sb, insert_times = create_modified_8760(clean_raw_data_sb, all_one_in_x_sb, t, custom_times=custom_times)
plot_modified8760s(modified8760_sb, insert_times, t)

## Inserting a min 1-in-X event into already modified 8760

Here, we will go through inserting another 1-in-X event into an already modified 8760, to showcase an 8760 timeseries with both a 1-in-X max temp and a 1-in-X min event inserted.

In [ ]:
%%time
# Calculating the 1-in-X min events,, and lower + upper CIs, for SB locations
sb_one_in_x_min = get_one_in_x(clean_raw_data_sb, one_in_x, event_duration, distr, 'min')

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Barbara locations
plot_retvals(sb_one_in_x_min['return_value'])

In [ ]:
%%time
# Calculating the 1-in-X min events,, and lower + upper CIs, for SB locations
sb_one_in_x_min = get_one_in_x(clean_raw_data_sb, one_in_x, event_duration, distr, 'min')

In [ ]:
# Plotting the 1-in-X values by simulation for Santa Barbara locations
plot_retvals(sb_one_in_x_min['return_value'])

In [ ]:
# Custom inserting events at hour=1000
updates = 1000
custom_times = set_custom_times(dummy_times, updates)

In [ ]:
# Insert a min 1-in-X event into already modified 8760
two_events_8760_sb, insert_times = create_modified_8760(
    clean_raw_data_sb,
    sb_one_in_x_min, 
    t, 
    extremes_type='min', 
    # custom_times=custom_times, # Comment this out if you'd like the back-end to auto-calculate the median time to insert these events
    custom_8760=modified8760_sb
)

In [ ]:
# Plot 8760 with 2 types of 1-in-X events inserted
plot_modified8760s(two_events_8760_sb, insert_times, t)

### Do the same for SC, but at hour=6000

In [ ]:
# Create `dummy_times` DataArray for SC
dummy_times = create_empty_da(all_one_in_x_sc['return_value'], keep_dims=['location', 'one_in_x'])

In [ ]:
# Modify event timing here
updates = 6000
custom_times = set_custom_times(dummy_times, updates)

In [ ]:
# Create and plot modified 8760 for SC
modified8760_sc, insert_times = create_modified_8760(clean_raw_data_sc, all_one_in_x_sc, t, custom_times=custom_times)
plot_modified8760s(modified8760_sc, insert_times, t)